# Deep Q Learning (DQN)
Hello again! If you find yourself here, then you're likely already familiar with Q learning and Q tables, and know how to implement these on your own games. Awesome!

If you followed along with the previous tutorial, you'll have also been asked to think about the limitations of Q learning, and Q tables in specific. Did you manage to come up with a solution? If not, this will be your last opportunity to think about it, because very soon we will learn the answer.

This tutorial will focus on one of the most important papers in modern reinforcement learning, called "Playing Atari with Deep Reinforcement Learning"<sup>[1](https://arxiv.org/pdf/1312.5602.pdf)</sup>. This paper came up with a way to break through the limitations of Q learning and build an agent capable of playing several atari games, some on a super human level!

We will first introduce why Q learning is insufficient, then proceed to build a minimalist implementation of deep Q learning, and finish off by adding accessories until we reach an implementation similar to that of the paper.

## Q learning's shortcomings
In the taxi problem of the last tutorial, we had to solve a problem which had 500 possible states and 6 possible actions. We could easily store this in memory, and computating the optimal table only cost us a few seconds at most.

Suppose we were to look at another game though, perhaps we'll try tic-tac-toe. In tic-tac-toe, we have 9 squares, each of which can house one of 3 values (cross, circle or empty), and at each turn we can attempt 9 actions (one for each square). In total, this would mean that we need to store (and iterate over) $3^9 \times 9 = 177.147$ different Q values. Significantly more than the taxi problem.

Similarly, if we were to try and learn pong on an $80\times 80$ grid, then we would have approximately $6400$ positions the ball could be in, and $50$ positions each of the paddles could be in, for a total of $16$ million potential states, each of which has 2 associated actions. Therefore we would have to store, and iterate over, approximately $32$ million Q values. As you might imagine, this gets out of hand really quickly, and makes it very impractical Q tables onto any real world problems.

## Enter: Deep Q Learning
Let us think back to why we constructed a Q table in the first place. The Q value of a state action pair was supposed to be an accurate measure for the "goodness" of an action $a$ in that specific state $s$. In order to arrive at all those values, we built a so called Q function $Q(s, a)$, which would take this state action pair as arguments, and return the appropriate Q value. Our table fulfilled the role of this function. Hence, if we were to ditch the idea of a table, we need something else that can act as a proxy to the "true" Q function; a function approximator if you will.

Now, as luck would have it, computer scientists have come up with an incredible suite of function approximators collectively known as **neural networks**. For the purpose of these tutorials (and even beyond), this is how you can think of a neural network: A smart computer sciency construct, that is very good at approximating functions.

This neural net will replace the function of our Q table, and we will be adapting our algorithm to work elegantly with this network.

# Building the Network
The first thing you should know about neural networks, is that they work by matrix multiplication. We won't go in great depth on how exactly neural networks work, but if you're interested, I would strongly recommend [this 3Blue1Brown video](https://www.youtube.com/watch?v=aircAruvnKk) that delves into the topic in decent detail. At the end of it, you'll have a good understanding of how a neural net functions.

This is important, because in the "normal" way of using a neural network, you tend to have a certain input (a 28 x 28 grid of pixels representing a number), and often multiple outputs (the labels 0 through 9).

In the field of reinforcement learning, and throughout this course, we'll be using this fact to turn the previous Q function defined as $Q(s, a) \rightarrow x$ towards one of the form $Q(s) \rightarrow A$, where $A$ is a vector and $A_a$ corresponds to the Q value of action $a$. For brevity, we will continue to use the $Q(s, a)$ notation; but beware that this really turns into something more akin to $Q(s)_a$.

## Network architectures
Network architectures used to really frustrate me when I started out in deep learning. There didn't seem to be any real rhyme or reason to what would be the optimal network design or a way to methodically figure out a good design. Later, I've come to appreciate that the actual network architecture you use is more of an art than it is a science. Nevertheless, there are a few general criteria you can use, which are summed up in 2 rules:

1. Convolutional layers *increase* in size;
2. Fully connected layers *decrease* in size.

One way to look at this is that convolutional layers seek to find complex patterns, so need to build on previously found patterns to find new, more intricate patterns (of which there are likely more than the simple ones). Fully connected layers, however, seek to abstractify all the patterns found, requiring fewer neurons at each subsequent layers.

Don't worry if you are unfamiliar with what exactly convolutional layers or fully connected layers are. In this series we will be sticking with fully connected layers as described in the 3Blue1Brown video above, because this allows us to focus on the algorithms of reinforcement learning, rather than on the deep learning aspect. Therefore we will not be looking to solve games based on their pixel representations just yet (for which we would need convolutional layers); although we might do so nearing the end of this series.

## Pytorch
In this tutorial series, we'll be making use of the neural net framework called "pytorch". Mostly because it is easier to debug than tensorflow and offers pythonic fine-grained control. That said, if you happen to already be familiar with tensorflow, feel free to use that instead. It will make no functional difference.

### Installation
The installation of pytorch is slightly more involved than most packages. The best way to do it is to go to their [get started page](https://pytorch.org/get-started/locally/) and follow their instructions. Alternatively, you can try running your code on [Google Colab](https://colab.research.google.com/) instead, where you should be able to run `import torch` right out of the gate.

### The basics
The core datatype of pytorch is the `torch.tensor` type, which is very similar (and based on) the `numpy.array` datatype. Perhaps the biggest differences are that they can be *moved between devices* (gpu and cpu), support keeping track of a *gradient graph* through the *autograd* functionality of pytorch, and support **lazy processing** (remembering the operations to do, but only actually doing them when they're needed). In fact, because they're so similar, tensors can actually use the exact same memory adress of a previously defined numpy array, making conversion really quick.

Aside from the core datatype of a tensor, the core *construct* of pytorch is the *torch.nn.Module* class. This could be anything from an activation function to an entire neural network, but ends up just being an object that *transforms* tensors you put in.

Right now, all these concepts still seem distant and disconnected, so let's take a break from the background for a while and start actually implementing the simple (fully connected) network we'll be using.

## Coding the network
First things first, we're going to have to import pytorch and define a network, which will inherit from the `torch.nn.Module` class (being a thing that tensors flow through).

In [9]:
# Importing everything we'll need in one spot.
import gym
import torch
import numpy
import matplotlib.pyplot as plt

In [2]:
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        ...

Here you'll notice that in the initialisation method we write, it's also important to actually initialize the `torch.nn.Module` class as well.

Remembering the general architecture of fully connected layers (called **linear layers** in pytorch) of decreasing the layer size, we'll construct a small, simple neural net with 4 layers: the input layer, a hidden layer of size 256, a hidden layer of size 64 and the output layer.

In [3]:
class Network(torch.nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        self.input = torch.nn.Linear(in_size, 256)
        self.hidden = torch.nn.Linear(256, 64)
        self.output = torch.nn.Linear(64, out_size)

Note how even though we wanted to define 4 layers, we only wrote 3 lines of code. How does this makes sense!?

Well, in pytorch, we don't really write layers, but instead write the _transformations_ that happen between layers. So the `self.input` transformation actually performs a vector transformation like this:
$$xA + B = y$$
where if we assume that $n = $`in_size`, then $x$ is the input vector of size $n$, $A$ is the weight matrix of size $n \times 256$, $B$ is a weight vector of size $256$, and $y$ is the output vector of size $256$.

Of course, we're not necessarily bound to $x$ being a vector (it may also be a multi-dimensional tensor), but the principle remains the same. This is also known as a **linear transformation**, which is where `torch.nn.Linear` gets its name.

If any of the mathematics here was confusing to you (matrices, vectors, tensors, GAH!), then we'd recommend you brush up on your linear algebra. Websites like [Khan academy](https://www.khanacademy.org/math/linear-algebra) offer high quality free courses.

The astute amongst you might have realized by now that although we technically have a network defined, we've never actually told our program what to do when a tensor passes through! In order to do this, we're going to have to implement a `forward` function on our network; so let's do that now. (It is worth mentioning that every `torch.nn.Module` requires the `forward` method to be implemented.)

In [4]:
class Network(torch.nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        self.input = torch.nn.Linear(in_size, 256)
        self.hidden = torch.nn.Linear(256, 64)
        self.output = torch.nn.Linear(64, out_size)
    
    def forward(self, t):
        t = self.input(t)   # Pass the tensor through the input transformation
        t = torch.nn.functional.relu(t)  # Apply the relu activation function
        t = self.hidden(t)  # Pass the tensor through the hidden transformation
        t = torch.nn.functional.relu(t)
        t = self.output(t)  # Pass the tensor through the output transformation
        return t

Of course we'll talk a little bit about the activation function, but this last step has actually completed our network!

The activation function is a concept from nature: We may not always want a neuron to "fire", sometimes we want it to "stay silent". In order to achieve this, we use a so-called **activation function** which uses a function to figure out if our neuron actually fires. In the case of a relu (REctified Linear Unit) activation function, we simply say that a neuron does not "fire" (its value is $0$) whenever its value is $\leq 0$.

Using the network becomes easy enough, simply calling it as `net(t)` whenever we want to transform a tensor `t`.

# Adapting Q learning
Okay, so all is well and good: we have the network to take the place of the Q table as our new function approximator... But we can't just drop it in and expect everything to work... can we?

No we can't. But what we can do is start thinking about the analogous actions we can take. Specifically, if we recall the training step done in normal Q learning, it would look something like this:
```python
# Pick an action
action = Qtable[state].argmax()
# Perform the action
next_state, reward, done, _ = env.step(action)
# Update the Q table
Qtable[state, action] = reward + gamma * Qtable[next_state].max()
```
The questions we thus need to answer are: How do we pick an action, and How do we update the network?

Well, picking an action might be fairly simple to solve. After all, we know we can get the Q values of all the available actions in a state by calling `network(state)`; so taking the argmax on that should do the trick.

As for updating the network, that's a whole other beast.

## Updating the network
As discussed before, a neural network gets better by improving its weights and biases (collectively known as **parameters**) in the direction of the gradient when comparing it to a loss function. The good news is that the gradient is calculated by pytorch on the fly, the bad news is that we have to introduce a loss function and a so called **optimizer** (function that actually performs gradient descent) in order to update the model, which adds some boilerplate code. So let's start setting up some of this code.

In [7]:
# Setting up the environment
env = gym.make('CartPole-v1')
num_observations = 4  # CartPole returns a Box(4,) object as the observation space, which means 4 floats in the shape of the tuple (4,).
num_actions = env.action_space.n  # CartPole has 2 discreet actions, left and right.

# Setting up the network. It is common practice to call the network the "policy" network
policy = Network(in_size=num_observations, out_size=num_actions)

# Set up the loss and optimizer functionality
optimizer = torch.optim.Adam(params=policy.parameters())  # We'll be using the Adam optimizer, but other options can also work.
loss_function = torch.nn.MSELoss()  # MSELoss stands for: Mean Squared Error Loss.

Having defined this functionality, let's also create some utility functions for choosing an action, calculating the loss, and updating the model.

In [8]:
def choose_action(state):
    return self.policy(state).argmax().item()  # Note the `item()` to turn a tensor that looks like [x] into the actual value of x.

def calculate_loss(state, action, reward, next_state, gamma):
    Qval = policy(state)[action]  # Get the actual Q value of this particular action in this particular state

    # Only assign a value to the next state if we're not already done.
    next_Qval = policy(next_state).max() if not done else 0

    Qgoal = torch.tensor(reward) + gamma * next_Qval  # Same as with the table

    return loss_function(Qval, Qgoal)  # Calculate the MSELoss between Qval and Qgoal

def update_model(loss):
    optim.zero_grad()  # Set the gradient to 0 everywhere
    loss.backward()    # Calculate the gradient based on the loss we just found
    optim.step()       # Update the parameters based on this loss.


## The new training loop
So after we've done all of this, our new training loop should look something like this:

In [ ]:
done = False
state = env.reset()
state = torch.tensor(state, dtype=torch.float32)
while not done:
    # Pick an action
    action = choose_action(state)
    
    # Perform the action
    next_state, reward, done, _ = env.step(action)
    # Turn the next state into a tensor so we can pass it to the network
    next_state = torch.tensor(next_state, dtype=torch.float32)
    
    loss = calculate_loss(state, action, reward, next_state, gamma)
    update_model(loss)

    state = next_state

The problem is, that this doesn't work! Don't get me wrong, the logic is sound; but the agent just won't learn. So let's clean up our code, put it all in a nice `Agent` class, and discuss how we might go about fixing this issue after.

## Code Cleanup
There are a few things we're going to change:

1. Notice how we have to transform every state into a tensor? That's ugly. Lets do that inside the forward function of our network.
2. There's actually a much nicer way to write down the network architecture by using `torch.nn.Sequential`, so while we're at it, we might as well change that.
3. All these loose functions can be emaglamated inside an Agent class which would look much better and be easier to maintain. Lets do that too.
4. Thusfar we haven't been concerned with whether we do computations on the gpu or cpu, but lets start putting things on the gpu from now on. (If there's a cuda device available anyway)

In [10]:
# (4)
# Lets start putting everything on the GPU if we can.
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

class Network(torch.nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        # (2)
        # Recall that fc stands for fully connected?
        # This is the fully connected (and in this case: only) component of the neural network
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_size, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, out_size)
        )
    
    def forward(self, t):
        # (1)
        # Should we get an array as input, change it to a tensor and move it to the appropriate device.
        if type(t) == np.array:
            t = torch.tensor(t, dtype=torch.float32).to(device)

        return self.fc(t)

# (3)
class Agent:

    def __init__(self, env):
        self.name = 'DQN'

        self.env = env
        self.action_space = env.action_space.name
        self.observation_space = env.observation_space.shape[0]
        
        self.gamma = 0.99

        # (4) Note that we're moving the network onto the appropriate device
        self.policy = Network(self.observation_space, self.action_space).to(device)
        self.optim = torch.optim.Adam(self.policy.parameters())
        self.loss = torch.nn.MSELoss()
    
    def choose_action(self, state):
        return self.policy(state).argmax().item()

    # The orchastrative part of this function comes from the Q learning tutorial
    def train(self, num_eps):
        scores = []
        for episode in range(num_eps):  # Run for a certain number of games
            print(f'running episode {episode}')
            score = 0
            done = False
            state = env.reset()
            while not done:
                action = choose_action(state)
                next_state, reward, done, _ = env.step(action)
                
                loss = calculate_loss(state, action, reward, next_state, gamma)
                update_model(loss)

                state = next_state
                score += reward
            
            scores.append(score)
    
    def _calculate_loss(self, state, action, reward, next_state, gamma):
        Qval = policy(state)[action]
        next_Qval = policy(next_state).max() if not done else 0

        Qgoal = torch.tensor(reward) + gamma * next_Qval

        return self.loss(Qval, Qgoal)

    def _update_model(loss):
        optim.zero_grad()
        Qloss.backward()
        optim.step()

Well... That was a big chunk of code. But most of it has been explained before, so I hope everyone managed to follow along. Lets start discussing the issues this code has.

# Fixing the algorithm


# References
1. [Playing Atari with Deep Reinforcement Learning, V. Mnih et all](https://arxiv.org/pdf/1312.5602.pdf)
